In [ ]:
cd /home/jovyan/capstone/GetOldTweets-python/

In [ ]:
import pandas as pd
import got3 as got
from pprint import pprint 

### Looking at JSON file to see if I can filter out tweets that are replies to other tweets. Also looking for geo and username fields.

**Update:**  Added functionality to skip replies. Found username field but couldn't find any location information (geo, coordinates, location).

In [ ]:
import urllib.request, urllib.parse, urllib.error,urllib.request,urllib.error,urllib.parse,json,re,datetime,sys,http.cookiejar
from got3 import models
from pyquery import PyQuery

In [ ]:
username = 'sfspca'
since = '2017-08-21'
until = '2017-08-23'

# max_tweets = 1

In [ ]:
# this is the beginning of getTweets, including some of the default parameters
tweetCriteria = got.manager.TweetCriteria().setUsername(username).setSince(since).setUntil(until)

receiveBuffer = None  # parameter passed to getTweets
bufferLength=100 # parameter passed to getTweets
proxy=None # parameter passed to getTweets
refreshCursor = ''
results = []
resultsAux = []
cookieJar = http.cookiejar.CookieJar()

In [ ]:
# this is all the functionality in getJsonResponse which is called inside getTweets
url = "https://twitter.com/i/search/timeline?f=tweets&q=%s&src=typd&%smax_position=%s"

urlGetData = ''
if hasattr(tweetCriteria, 'username'):
	urlGetData += ' from:' + tweetCriteria.username
	
if hasattr(tweetCriteria, 'since'):
	urlGetData += ' since:' + tweetCriteria.since
	
if hasattr(tweetCriteria, 'until'):
	urlGetData += ' until:' + tweetCriteria.until
	
if hasattr(tweetCriteria, 'querySearch'):
	urlGetData += ' ' + tweetCriteria.querySearch
	
if hasattr(tweetCriteria, 'lang'):
	urlLang = 'lang=' + tweetCriteria.lang + '&'
else:
	urlLang = ''
url = url % (urllib.parse.quote(urlGetData), urlLang, refreshCursor)
print(url)
headers = [
	('Host', "twitter.com"),
	('User-Agent', "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"),
	('Accept', "application/json, text/javascript, */*; q=0.01"),
	('Accept-Language', "de,en-US;q=0.7,en;q=0.3"),
	('X-Requested-With', "XMLHttpRequest"),
	('Referer', url),
	('Connection', "keep-alive")
]

if proxy:
	opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
else:
	opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
opener.addheaders = headers

response = opener.open(url)
jsonResponse = response.read()

dataJson = json.loads(jsonResponse.decode())

In [ ]:
dataJson['items_html'].keys()

In [ ]:
# pprint(dataJson)

In [ ]:
# how to see if tweet is a reply
tweetPQ("div.tweet").attr("data-is-reply-to")

In [ ]:
# how to get username from tweet
tweetPQ("div.tweet").attr("data-screen-name")

### Now that I got json file, look at how getTweets gets the attributes

In [ ]:
len(dataJson['items_html'].strip())

In [ ]:
refreshCursor = dataJson['min_position']
tweets = PyQuery(dataJson['items_html'])('div.js-stream-tweet')

for tweetHTML in tweets:
    tweetPQ = PyQuery(tweetHTML)
    tweet = models.Tweet()
    break

### Create data frame to see what gets outputted in a dataframe

In [ ]:
tweet_list = got.manager.TweetManager.getTweets(tweetCriteria)

from collections import defaultdict
d = defaultdict(list)
attributes = ['date', 'favorites', 'retweets', 'hashtags', 'geo', 'id',
                  'mentions', 'text', 'urls', 'username', 'permalink', 'author_id']
for tweet in tweet_list:
    for att in attributes:
        d[att].append(eval("tweet." + att))

pd.DataFrame(d)

In [ ]:
# ADJUSTMENT FOR UNTIL NOT BEING INCLUSIVE, SO START EACH MONTH WITH END OF PREVIOUS MONTH
#     years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
#     months = [('01', '31'), ('02', '28'), ('03', '31'), ('04', '30'), ('05', '31'), ('06', '30'),
#               ('07', '31'), ('08', '31'), ('09', '30'), ('10', '31'), ('11', '30'), ('12', '31')]
#     total = 0
#     for year in years:
#         for month in months:
#             if year == '2008' and month[0] == '01':
#                 since = year + '-' + month[0] + '-01'
#             else:
#                 since = until
#             until = year + '-' + month[0] + '-' + month[1]
#             print(since, until)